In [1]:
import os
import glob
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

import tensorflow as tf
import yolo.config as cfg
from utils.help import say

Using TensorFlow backend.


In [2]:
sess = tf.Session()
K.set_session(sess)

K.set_image_dim_ordering('th')
weights_path = 'yolo-tiny-origin.h5'
is_freeze = True
verbalise = True

In [3]:
def make_yolotiny_network(is_freeze=True):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(3,448,448),padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(64,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(128,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(256,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(512,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))
    
    return model

In [4]:
def make_bottleneck_network(model):
    model.add(Dense(256, input_shape=model.output_shape, 
                    kernel_initializer='random_uniform',
                    bias_initializer='zeros'))
    model.add(Dense(4096,
                    kernel_initializer='random_uniform',
                    bias_initializer='zeros'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470,
                    kernel_initializer='random_uniform',
                    bias_initializer='zeros'))
    
    return model

In [5]:
model = make_yolotiny_network()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [6]:
from yolo.training_v1 import darkeras_loss, _TRAINER
from yolo.datacenter.data import shuffle

### 처음에는 랜덤 이니셜라이즈가 된걸로 보임

In [7]:
sess.run(tf.global_variables_initializer())

inp_x = model.input
net_out = model.output

conv1 = model.layers[0]
dense_last = model.layers[-1]
print("conv1 : ")
print(sess.run(conv1.weights[0])[0][0][0])
print("danse last : ")
print(sess.run(dense_last.weights[0])[0][0:10])

conv1 : 
[-0.17770042 -0.09734315  0.04980351  0.08971766  0.03622699  0.03267483
  0.02295871  0.12204838 -0.00864786  0.10797963 -0.12393852 -0.08594502
  0.12667552  0.06479004  0.1661678   0.1166352 ]
danse last : 
[-0.02657805  0.02436714  0.00917208  0.01819196 -0.01418575 -0.00403487
  0.00861444 -0.01984378 -0.02592226  0.02687271]


### 강제로 global initializer 를 시키면?

In [8]:
model.trainable_weights

[<tf.Variable 'dense_1/kernel:0' shape=(50176, 256) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(256, 4096) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'dense_3/kernel:0' shape=(4096, 1470) dtype=float32_ref>,
 <tf.Variable 'dense_3/bias:0' shape=(1470,) dtype=float32_ref>]

In [9]:
inp_x = model.input
net_out = model.output


say("Building {} loss function".format(cfg.model_name), verbalise=verbalise)
loss_ph, loss_op = darkeras_loss(net_out)
say("Building {} train optimizer".format(cfg.model_name), verbalise=verbalise)
optimizer = _TRAINER[cfg.trainer](cfg.lr)
gradients = optimizer.compute_gradients(loss_op, var_list=model.trainable_weights)
train_op = optimizer.apply_gradients(gradients)

sess.run(tf.global_variables_initializer())

conv1 = model.layers[0]
dense_last = model.layers[-1]
print("conv1 : ")
print(sess.run(conv1.weights[0])[0][0][0])
print("danse last : ")
print(sess.run(dense_last.weights[0])[0][0:10])

['Building yolo-tiny loss function']
Building yolo-tiny loss
['Building yolo-tiny train optimizer']
conv1 : 
[-0.1152859   0.1666998  -0.12276138  0.11065465 -0.18027841 -0.1055052
  0.08193085 -0.01095307  0.0993349  -0.08545657  0.08402526  0.02083692
  0.16605207  0.11977512 -0.11258174  0.09514982]
danse last : 
[ 0.00687876  0.01418922 -0.02760005  0.01720214 -0.020932    0.00023801
  0.02182424  0.02264647 -0.00902974 -0.01353666]


In [10]:
model.load_weights(weights_path)
say("Setting weigths : {}",format(weights_path), verbalise=verbalise)

pop_layer = model.pop() # dense_25
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # leakyrelu_34
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense_24
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense 23
say("{} layer poped".format(pop_layer), verbalise=verbalise)

model.summary()
print(model.output_shape)

['Setting weigths : {}', 'yolo-tiny-origin.h5']
['None layer poped']
['None layer poped']
['None layer poped']
['None layer poped']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
__________

In [11]:
model = make_bottleneck_network(model)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [12]:
model.trainable_weights

[<tf.Variable 'dense_4/kernel:0' shape=(50176, 256) dtype=float32_ref>,
 <tf.Variable 'dense_4/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(256, 4096) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'dense_6/kernel:0' shape=(4096, 1470) dtype=float32_ref>,
 <tf.Variable 'dense_6/bias:0' shape=(1470,) dtype=float32_ref>]

In [ ]:
# sess.run(tf.local_variables_initializer(model.trainable_weights))

In [17]:
freeze_layer_weights = None
trainable_layer_weights = None

batches = shuffle()
for i, (x_batch, datum) in enumerate(batches):
    train_feed_dict = {
       loss_ph[key]:datum[key] for key in loss_ph 
    }
    train_feed_dict[inp_x] = x_batch
    # print("feed_dict.keys() : ", len(train_feed_dict.keys()), train_feed_dict.keys())
    fetches = [train_op, loss_op] 
    fetched = sess.run(fetches, feed_dict=train_feed_dict)
    
    loss_val = fetched[1]
    say("step {} - loss {}".format(i, loss_val), verbalise=True)
    
    conv1 = model.layers[0]
    dense_last = model.layers[-1]
    if freeze_layer_weights is None:
        freeze_layer_weights = sess.run(conv1.weights[0])
        trainable_layer_weights = sess.run(dense_last.weights[0])
    else:
        print("conv1 : ", freeze_layer_weights == sess.run(conv1.weights[0]))
        # print(sess.run(conv1.weights[0])[0][0][0])
        print("danse last : ", dense_last.name, trainable_layer_weights == sess.run(dense_last.weights[0]))
        # print(sess.run(dense_last.weights[0])[0][0:10])
    
    if i == 1:
        save_step_weigths_path = 'layerfreeze-step{}.h5'.format(i)
        model.save_weights(save_step_weigths_path)
        print("Saved weigths : ", save_step_weigths_path)
    if i % 310 == 0:
        model.save_weights('layerfreeze-epoch{}.h5'.format(i//310))
        say("Save weights : ", 'layerfreeze-epoch{}.h5'.format(i//310), verbalise=verbalise)

Dataset of 9963 instance(s)
shuffle index :  [2272 1244   83 ..., 6689 4642 6164]
['step 0 - loss 2.8707242012023926']
['Save weights : ', 'layerfreeze-epoch0.h5']
['step 1 - loss 2.407315731048584']
conv1 :  [[[[ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]
   [ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]
   [ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]]

  [[ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]
   [ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]
   [ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]]

  [[ True  True  True  True  True  True  True  True  True  True  True  True
     True  True  True  True]
   [ True  True  True  True  True  True  True  True 

AttributeError: 'NoneType' object has no attribute 'all'

In [ ]:
sess.close()